In [0]:
# !pip install tensorflow-gpu

In [0]:
# from tensorflow.python.client import device_lib
# print('Show system RAM memory\n\n')
# !cat /proc/meminfo | egrep 'MemTotal*'
# print('\nShow devices : \n'+str(device_lib.list_local_devices()))

In [0]:
import numpy as np
import tensorflow as tf

# print("TF version : ", tf.__version__)
# print("TF-GPU available? ", tf.test.is_gpu_available())

In [0]:
# hyperparameters
learning_rate = 0.001
num_epochs = 10
batch_size = 50

dataset_id = 1          # 1 - MNIST, 2 - Cifar10
if dataset_id == 1:
  img_size = [28,28,1]
else:
  img_size = [32,32,3]
num_fc1_units = 1000
num_classes = 10
num_filter_layer1 = 32
num_filter_layer2 = 64

In [0]:
def data_mnist():
  from tensorflow.examples.tutorials.mnist import input_data
  mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
  train_x = np.reshape(mnist.train.images, (mnist.train.labels.shape[0], img_size[0], img_size[1], img_size[2]))
  train_y = mnist.train.labels
  test_x = np.reshape(mnist.test.images, (mnist.test.labels.shape[0], img_size[0], img_size[1], img_size[2]))
  test_y = mnist.test.labels
  return (train_x, train_y, test_x, test_y)

In [0]:
def data_cifar10():
  (train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar10.load_data()
  # preprocessing the input image features
  temp1 = np.reshape(train_x,(train_x.shape[0],img_size[0]*img_size[1]*img_size[2]))
  temp2 = np.reshape(test_x,(test_x.shape[0],img_size[0]*img_size[1]*img_size[2]))
  mean = np.mean(temp1,axis=0)
  stddev = np.std(temp1,axis=0)
  temp1 = (temp1 - mean)/stddev
  temp2 = (temp2 - mean)/stddev
  train_x = np.reshape(temp1,(train_x.shape[0],img_size[0],img_size[1],img_size[2]))
  test_x = np.reshape(temp2,(test_x.shape[0],img_size[0],img_size[1],img_size[2]))
  
  # converting labels to one-hot encoding
  temp1 = np.zeros((train_y.shape[0],10),dtype=np.float32)
  temp2 = np.zeros((test_y.shape[0],10),dtype=np.float32)
  temp1[np.arange(train_y.shape[0]),np.reshape(train_y,(train_y.shape[0]))] = 1.0
  temp2[np.arange(test_y.shape[0]),np.reshape(test_y,(test_y.shape[0]))] = 1.0
  return (train_x, temp1, test_x, temp2)

In [0]:
train_x, train_y, test_x, test_y = data_mnist() if dataset_id == 1 else data.cifar10()
x = tf.placeholder(tf.float32, [None, img_size[0], img_size[1], img_size[2]], name="data")
y = tf.placeholder(tf.float32, [None, num_classes], name="labels")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
def weight_variable(shape, mu=0, sigma=1.0):
	weight = tf.truncated_normal(shape=shape, mean=mu, stddev=sigma, dtype=tf.float32)
	return tf.Variable(weight)

In [0]:
def bias_variable(shape, mu=0, sigma=1.0):
  bias = tf.truncated_normal(shape=shape, mean=mu, stddev=sigma, dtype=tf.float32)
  return tf.Variable(bias)

In [0]:
def batch_normalization_fc(input_tensor):
  variance_epsilon = 1e-10
  input_tensor = tf.cast(input_tensor, dtype=tf.float32)
  moments = tf.nn.moments(input_tensor, axes=0)     # moment return mean and variance of the tensor
  norm_tensor = tf.divide(tf.subtract(input_tensor, moments[0]), tf.sqrt(tf.add(moments[1], variance_epsilon)))
  gamma = weight_variable(shape=tf.shape(moments[0]))
  beta = weight_variable(shape=tf.shape(moments[0]))
  out_tensor = tf.add(tf.multiply(norm_tensor, gamma), beta)
  return out_tensor

In [0]:
def batch_normalization_conv(input_tensor):
  variance_epsilon = 1e-10
  input_tensor = tf.cast(input_tensor, dtype=tf.float32)
  moments = tf.nn.moments(input_tensor, axes=0)
  norm_tensor = tf.divide(tf.subtract(input_tensor, moments[0]), tf.sqrt(tf.add(moments[1], variance_epsilon)))
  gamma = weight_variable(shape=tf.shape(moments[0]))
  beta = weight_variable(shape=tf.shape(moments[0]))
#   out_tensor = tf.nn.batch_normalization(x=input_tensor,mean=moments[0],variance=moments[1],offset=beta,scale=gamma,variance_epsilon=1e-10)
  out_tensor = tf.add(tf.multiply(norm_tensor, gamma), beta)
  return out_tensor

In [0]:
def group_norm(input_tensor, G=32, eps=1e-5, scope='group_norm') :
    with tf.variable_scope(scope) :
        N, H, W, C = input_tensor.get_shape().as_list()
        G = min(G, C)

        input_tensor = tf.reshape(input_tensor, [N, H, W, G, C // G])
        mean, var = tf.nn.moments(input_tensor, [1, 2, 4], keep_dims=True)
        input_tensor = (input_tensor - mean) / tf.sqrt(var + eps)

        gamma = tf.get_variable('gamma', [1, 1, 1, C], initializer=tf.constant_initializer(1.0))
        beta = tf.get_variable('beta', [1, 1, 1, C], initializer=tf.constant_initializer(0.0))
        input_tensor = tf.reshape(input_tensor, [N, H, W, C]) * gamma + beta

    return input_tensor

The below function is taken from https://github.com/tensorpack/tensorpack/blob/79148350eabd6800133a49b101eea4c56e78e4e8/examples/ImageNetModels/vgg16.py#L19-L46

In [0]:
def GroupNorm(x, group, gamma_initializer=tf.constant_initializer(1.)):
    """
    https://arxiv.org/abs/1803.08494
    """
    x = tf.transpose(x, [0,3,1,2])
    shape = x.get_shape().as_list()
    ndims = len(shape)
    assert ndims == 4, shape
    chan = shape[1]
    assert chan % group == 0, chan
    group_size = chan // group

    orig_shape = tf.shape(x)
    h, w = orig_shape[2], orig_shape[3]

    x = tf.reshape(x, tf.stack([-1, group, group_size, h, w]))

    mean, var = tf.nn.moments(x, [2, 3, 4], keep_dims=True)

    new_shape = [1, group, group_size, 1, 1]

#     beta1 = tf.get_variable('beta1', [chan], initializer=tf.constant_initializer())
#     beta1 = tf.reshape(beta1, new_shape)

#     gamma1 = tf.get_variable('gamma1', [chan], initializer=gamma_initializer)
#     gamma1 = tf.reshape(gamma1, new_shape)
    gamma = weight_variable(shape=new_shape)
    beta = weight_variable(shape=new_shape)

    out = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-5, name='output')
    out = tf.reshape(out, orig_shape, name='output')
    return tf.transpose(out, [0,2,3,1])

In [0]:
def create_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
    # setup the filter input shape for tf.nn.conv_2d
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels, num_filters]

    # initialise weights and bias for the filter
    weights = tf.Variable(tf.truncated_normal(shape=conv_filt_shape, stddev=0.03), name=name+'_W')
    bias = tf.Variable(tf.truncated_normal(shape=[num_filters]), name=name+'_b')

    # setup the convolutional layer operation
    out_layer = tf.nn.conv2d(input=input_data, filter=weights, strides=[1, 1, 1, 1], padding='SAME') + bias

#     out_layer = batch_normalization_conv(out_layer)
#     out_layer = tf.contrib.layers.batch_norm(out_layer)
    out_layer = GroupNorm(out_layer,group=16)

    # apply a ReLU non-linear activation
    out_layer = tf.nn.relu(out_layer)

    # now perform max pooling
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    pool_strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize=ksize, strides=pool_strides, padding='SAME')

    return out_layer

In [0]:
def get_next_batch(step, batch_size):
    return train_x[step*batch_size:(step+1)*batch_size,:,:,:], train_y[step*batch_size:(step+1)*batch_size,:]

In [0]:
def create_fc_layer(input_tensor, num_input_units, num_output_units):
  weight = weight_variable(shape=[num_input_units, num_output_units])
  bias = bias_variable(shape=[num_output_units])
  output_tensor = tf.add(tf.matmul(input_tensor, weight), bias)
#   output_tensor = batch_normalization_fc(output_tensor)
  output_tensor = tf.contrib.layers.batch_norm(output_tensor)
  return output_tensor

In [0]:
# create some convolutional layers
layer1 = create_conv_layer(x, img_size[2], num_filter_layer1, [5, 5], [2, 2], name='layer1')
mod_img_size = [img_size[0]//2, img_size[1]//2, img_size[2]]
layer2 = create_conv_layer(layer1, num_filter_layer1, num_filter_layer2, [5, 5], [2, 2], name='layer2')
mod_img_size = [mod_img_size[0]//2, mod_img_size[1]//2, mod_img_size[2]]

flattened = tf.reshape(layer2, [-1, mod_img_size[0]*mod_img_size[1]*num_filter_layer2])

# setup some weights and bias values for this layer, then activate with ReLU
dense_layer_1 = tf.nn.relu(create_fc_layer(input_tensor=flattened, num_input_units=mod_img_size[0]*mod_img_size[1]*num_filter_layer2, num_output_units=num_fc1_units))
# another layer with softmax activations
dense_layer_2 = create_fc_layer(input_tensor=dense_layer_1, num_input_units=num_fc1_units, num_output_units=num_classes)
y_predicted = tf.nn.softmax(dense_layer_2)

with tf.name_scope("loss_value"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=dense_layer_2, labels=y))
tf.summary.scalar('cross_entropy', cross_entropy)

# add an optimizer
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

# define an accuracy assessment operation
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_predicted, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

# setup recording variables
merged_summary = tf.summary.merge_all()
# test_writer = tf.summary.FileWriter('/tmp/mnist_demo/test/1')

In [0]:
iteration_step = 1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     train_writer = tf.summary.FileWriter('./logs/', sess.graph)
    total_batch = int(train_y.shape[0]/batch_size)
    
    for epoch in range(num_epochs):
        for itr in range(total_batch):
#             if iteration_step % 10 == 0:
#                 test_accuracy, loss_value = sess.run([accuracy, cross_entropy], feed_dict={x: mnist.test.images, y: mnist.test.labels})
#                 test_writer.add_summary(summary, iteration_step)
#                 print('test accuracy at step %s: %s' % (iteration_step, test_accuracy))

            batch_x, batch_y = get_next_batch(step=itr, batch_size=batch_size)
            _ = sess.run([optimizer], feed_dict={x:batch_x, y:batch_y})
#             train_writer.add_summary(summary, iteration_step)
            iteration_step += 1
#             print(iteration_step)

    print("\nTraining complete!")
    print("final test accuracy", sess.run(accuracy, feed_dict={x: test_x, y: test_y}))


Training complete!
final test accuracy 0.9923
